<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="25%"><img src="media/decartes.jpg"
alt="DeCART Icon" width="128" height="171"><br>
</td>
<td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">DeCART Summer School<br>
for<br>
Biomedical Data Science</font></h1></td>
<td valign="center" align="center" width="25%"><img
src="media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


# Working with Textual Data
#### &copy; Brian E. Chapman, Ph.D.


### To get the latest version of this lesson, execute the cell below

In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

In [16]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [8]:
rad_data = pd.read_sql("""SELECT * FROM noteevents LIMIT 10000""", conn)
rad_data.head()
set(rad_data["category"])

{'DISCHARGE_SUMMARY', 'MD Notes', 'Nursing/Other', 'RADIOLOGY_REPORT'}

In [17]:
rad_data = pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [9]:
rad_data.shape

(10000, 14)

## Section Splitting

<img src="http://medicine.utah.edu/dbmi/images/faculty/Chapman,Wendy_240x360.jpg"
alt="Wendy Chapman" width="128">

Clinical reports are often divided into sections. Physicians are taught to write their reports in the SOAP format: subjective information, objective information, assessment, and plan, and sections often reflect this process. Information each setion may be formatted differently. For example, subjective information is often narrative, telling a story of the patient's description of their past medical history and their current issues. The objective section may be much more telegraphic with shortened descriptions of measurements like blood pressure and heart rate, such as "120/80 88". You may need different NLP techniques to accurately extract information out of different sections. Sentence splitting, for example, may be different in subjective versus objective sections. Radiology reports are not in the SOAP format, but section identification can still be very important, because in these reports the radiologist describes what she sees on the radiographic image then sometimes summarizes her impression of what the observations may represent, such as "the infiltrate could be consistent with pneumonia." For some tasks, you may only be interested in the impression, which has more weight than the reasoning that occurs in the Findings section.

## Exercise

Write a function that returns the impression section of a report. The function will take two arguments:

1. The text to search for the impression section is the first positional argument.
1. A list of phrases to use to identify the impression section.

Not every report will have a section labeled "IMPRESSION". You can consider "INTERPRETATION" and "CONCLUSIONS" as synonyms for "IMPRESSION"

If you cannot identify the impresison section (or equivalent), return an empty string.

#### Hints

* Use a for loop  to iterate across
* Use the find method of a string to identify where in the string the impression section occurs
* Use slicing to grab the impression section


In [18]:
txt1 = rad_data.loc[1,'text']
print(txt1)




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [19]:
txt10 = rad_data.loc[10,'text']
print(txt10)




     DATE: [**3264-8-16**] 1:53 PM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 10701**]
     Reason: 68 yo M CHF, SOB                                            
     Admitting Diagnosis: CONGESTIVE HEART FAILURE
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
        68 year old man with sob, hx of chf, cad s/p CABG                             
       
     REASON FOR THIS EXAMINATION:
      68 yo M CHF, SOB                                                                
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Shortness of breath with history of CHF and CAD.
     
     Comparison [**3264-8-14**].
     
     FINDINGS:  Patient is status post CABG and multiple sternal wires appear
     fractured, not changed since the previous exam.  Cardiac and 

In [10]:
help(find_impression)

NameError: name 'find_impression' is not defined

In [21]:

def find_impression(txt, search_terms=["IMPRESSION", "INTERPRETATION", "CONCLUSION"]):
    for term in search_terms:
        ind = txt.find(term)
        if ind != -1:
            return txt[ind:]
         
    return ""

find_impression(txt10)


'IMPRESSION:\n     1. Persistence of interval slight worsening of degree of patchy opacity within\n     the lingula.  Continued follow up is recommended to exclude pneumonia.\n     2. Persistent mild CHF.\n\n'

## Exercise 

Use ``apply()`` to create a new column in the ``rad_data`` named "IMPRESSIONS" with the string identified with ``find_impression()``.

In [23]:
rad_data["IMPRESSIONS"] = rad_data["text"].apply(find_impression)
rad_data.head()

,subject_id,hadm_id,text,IMPRESSIONS
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,IMPRESSION: Stable appearance of right pariet...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,IMPRESSION:\n \n Cardiomegaly and mild...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,IMPRESSION:\n \n Marked improvement in...


## How else might we need to break up a report?

* How would I break a report into sentences?
* How would I break a report into words?

## Group Exercise

### Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame
* Use TextBlob to get words

In [25]:
import numpy as np

In [35]:
def do_math(func):
    return[func(i) for i in np.arange(-np.pi,np.pi,0.5)]

In [36]:
do_math(np.cos)

[-1.0,
 -0.8775825618903726,
 -0.5403023058681397,
 -0.07073720166770278,
 0.4161468365471425,
 0.8011436155469338,
 0.9899924966004455,
 0.9364566872907963,
 0.6536436208636118,
 0.21079579943077958,
 -0.28366218546322636,
 -0.7086697742912601,
 -0.9601702866503661]

In [37]:
do_math(lambda x:2*x)

[-6.283185307179586,
 -5.283185307179586,
 -4.283185307179586,
 -3.2831853071795862,
 -2.2831853071795862,
 -1.2831853071795862,
 -0.28318530717958623,
 0.7168146928204138,
 1.7168146928204138,
 2.7168146928204138,
 3.7168146928204138,
 4.716814692820414,
 5.716814692820414]